In [1]:
#Dependencies
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [2]:
#Initialize browser
#Change chromedriver path when uploading to server

executable_path = {'executable_path': r"C:\Users\slapp\Desktop\Misc_class\chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)


In [3]:
#define initial url and direct browser
url = 'https://www.indeed.com/jobs?q=Data+Scientist&explvl=entry_level&fromage=3'


In [4]:
browser.visit(url)

In [5]:
#Make some soup
html = browser.html
soup = BeautifulSoup(html, 'lxml')
browser.quit()

In [6]:
#Scrap all postings on page into a list. Run through loop to click on next page
postings = soup.find_all( class_= 'jobsearch-SerpJobCard')

In [7]:
postings[0]

<div class="jobsearch-SerpJobCard unifiedRow row result clickcard" data-ci="350917459" data-empn="6281437093694551" data-jk="992a4f3b6125f8ec" id="pj_992a4f3b6125f8ec">
<style>
.jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}
</style>
<div class="jobcard_logo">
<a href="/cmp/IT-Partners,inc" onmousedown="this.href = appendParamsOnce(this.href, 'tk=1ebfe74683aci000&amp;campaignid=femp&amp;from=femp');" rel="noopener" target="_blank">
<img alt="IT Partners,Inc" src="https://d2q79iu7y748jz.cloudfront.net/s/_logo/888f6ecf0496632573fd684426fafd90"/>
</a>
</div>
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0ApfOu4Ek3LFcwJt5wzewEn0x2OMPseiW0RcmhgCc-1KgKXzvd4tCNB2UpaGIfi68ZaT0243btKe4na-e8GDtn9bkBRGY0EBMCUfc2YXaGD4rjuXLtZWt_nEGcaazFOMeEQqzQRa14bj4WTBx1Z89oJ5Ca1Tn5GqEc5RvZ-V_9dNrnAy7ID-40Jr7zb-d50TlFy_6lnw2-AEFZgsd6JRRItiGNvNhmKok8fVORMSSCuRD99theI-TKQQNgUXDAwwvn_3mFW7TOQhoIr-ZKTECtX

In [8]:
len(postings)

16

In [9]:
#Scrape url and add domaine
f"https://www.indeed.com{postings[0].find('h2').find(class_ = 'jobtitle')['href']}"


'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0ApfOu4Ek3LFcwJt5wzewEn0x2OMPseiW0RcmhgCc-1KgKXzvd4tCNB2UpaGIfi68ZaT0243btKe4na-e8GDtn9bkBRGY0EBMCUfc2YXaGD4rjuXLtZWt_nEGcaazFOMeEQqzQRa14bj4WTBx1Z89oJ5Ca1Tn5GqEc5RvZ-V_9dNrnAy7ID-40Jr7zb-d50TlFy_6lnw2-AEFZgsd6JRRItiGNvNhmKok8fVORMSSCuRD99theI-TKQQNgUXDAwwvn_3mFW7TOQhoIr-ZKTECtXhq-cpLF065qSjgw6hZNDRQb_-YMAnkUCEDR8ab3IXSR_AZbUoC_rPSC798MjeNK3Q83Z7bDGsouJ_Hqx4-jU3P4_fAIWQGdHWEom9---jNmX9NEFHg-DPiNpMAtnlSFkLaX6kQ1Bfrk=&p=0&fvj=0&vjs=3'

In [10]:
#Scrape job title
postings[0].find('h2').find(class_ = 'jobtitle').text.replace('\n', '')


'Data Scientist'

In [11]:
#scrape company name
postings[0].find(class_ = 'company').text.replace('\n', '')

'IT Partners,Inc'

In [12]:
#scrape location (split into city and state later)
location = postings[0].find(class_ = 'recJobLoc')['data-rc-loc']

In [13]:
#splitting location into city and state. try except to include remote work
cities = []
states = []
for x in range(0, len(location)):
    citystate = location[x].split(', ')
    cities.append(citystate[0])
    try:
        states.append(citystate[1])
    except IndexError:
        states.append(citystate[0])
    #print(cities[x] + states[x])

In [14]:
#scrape date added (current day - "days posted ago")
postings[0].find(class_='date').text

'3 days ago'

In [15]:
#Convert result to int
try:
    testdate = int(postings[0].find(class_='date').text.split(' ')[0])
except ValueError:
    testdate = 0

In [16]:
#subtract from current date w/ date time
d = datetime.today() - timedelta(days=testdate)
d

datetime.datetime(2020, 6, 19, 20, 56, 0, 381420)

In [17]:
#scrape salary, add null if not included
salaryrange = []
try:
    salaryrange.append(postings[1].find(class_ = 'salaryText').text.replace('\n', ''))
except AttributeError:
    salaryrange.append('NaN')
salaryrange

['NaN']

In [18]:
#scrape job abstract
postings[0].find(class_ = 'summary').find_all('li')

[<li>In your role you will work with various teams to identify, manage, research, develop, and present <b>data</b> analytics projects and solutions.</li>]

In [19]:
postings[0].find(class_ = 'summary').find_all('li')

[<li>In your role you will work with various teams to identify, manage, research, develop, and present <b>data</b> analytics projects and solutions.</li>]

In [20]:
#Collect all above data for each entry with a loop
post_url = []
job_title = []
company = [] 
city = []
state = []
date_added = []
salaryrange = []

        
for x in range(0, len(postings)):
    post_url.append(f"https://www.indeed.com{postings[x].find('h2').find(class_ = 'jobtitle')['href']}")
    job_title.append(postings[x].find('h2').find(class_ = 'jobtitle').text.replace('\n', ''))
    company.append(postings[x].find(class_ = 'company').text.replace('\n', ''))
    location = postings[x].find(class_ = 'recJobLoc')['data-rc-loc'].split(', ')
    city.append(location[0])
    try:
        state.append(location[1])
    except IndexError:
        state.append(location[0])
    try:
        temp_date = int(postings[x].find(class_='date').text.split(' ')[0])
    except ValueError:
        temp_date = 0
    date_added.append(datetime.today() - timedelta(days=testdate))
    try:
        salaryrange.append(postings[x].find(class_ = 'salaryText').text.replace('\n', ''))
    except AttributeError:
        salaryrange.append('NaN')

In [21]:
date_added

[datetime.datetime(2020, 6, 19, 20, 56, 0, 499085),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 512051),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 517038),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 520030),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 531011),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 535990),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 540978),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 545961),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 547954),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 550945),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 558929),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 568900),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 576878),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 579874),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 588852),
 datetime.datetime(2020, 6, 19, 20, 56, 0, 591843)]

In [22]:
#loop through pages

In [36]:
base_url = 'https://www.indeed.com/jobs?q=Data+Scientist&explvl=entry_level&fromage=3&start='
post_url = []
job_title = []
company = [] 
city = []
state = []
date_added = []
salaryrange = []


for x in range(0, 100, 10):
    test_url = f"{base_url}{x}"
    page = requests.get(test_url).content
    soup = BeautifulSoup(page)
    for x in range(0, len(postings)):
        post_url.append(f"https://www.indeed.com{postings[x].find('h2').find(class_ = 'jobtitle')['href']}")
        job_title.append(postings[x].find('h2').find(class_ = 'jobtitle').text.replace('\n', ''))
        company.append(postings[x].find(class_ = 'company').text.replace('\n', ''))
        location = postings[x].find(class_ = 'recJobLoc')['data-rc-loc'].split(', ')
        city.append(location[0])
        try:
            state.append(location[1])
        except IndexError:
            state.append(location[0])
        try:
            temp_date = int(postings[x].find(class_='date').text.split(' ')[0])
        except ValueError:
            temp_date = 0
        date_added.append(datetime.today() - timedelta(days=testdate))
        try:
            salaryrange.append(postings[x].find(class_ = 'salaryText').text.replace('\n', ''))
        except AttributeError:
            salaryrange.append('NaN')



In [39]:
salaryrange

['NaN',
 'NaN',
 '$90,000 - $100,000 a year',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 '$65 - $75 an hour',
 'NaN',
 'NaN',
 '$100,000 - $120,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$250,000 - $375,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$90,000 - $100,000 a year',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 '$65 - $75 an hour',
 'NaN',
 'NaN',
 '$100,000 - $120,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$250,000 - $375,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$90,000 - $100,000 a year',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 '$65 - $75 an hour',
 'NaN',
 'NaN',
 '$100,000 - $120,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$250,000 - $375,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$90,000 - $100,000 a year',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 '$65 - $75 an hour',
 'NaN',
 'NaN',
 '$100,000 - $120,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$250,000 - $375,000 a year',
 'NaN',
 'NaN',
 'NaN',
 '$90,000 - $100,000 a year',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 '$65 - $75 an hour',
 'NaN',
 'NaN',
 '$100,000 - $120,000 a year',
 'NaN',
 'NaN